In [1]:
from lxml import html
import requests
from collections import OrderedDict
import json
import argparse
import pandas as pd
import csv

def parse(input_file):
    restaurants = []
    for line in input_file:
        url = line
        response = requests.get(url).text
        parser = html.fromstring(response)
	
        XPATH_RATING = '//div[@id="ratingFilter"]//ul//li'

        XPATH_TRAVELER = '//div[@class="col segment extraWidth"]//ul//li'
        XPATH_TIME = '//div[@class="col season extraWidth"]//ul/li'
        XPATH_LANGUAGE = '//div[@class="col language extraWidth"]//ul/li'
#        XPATH_DETAILS = '//div[contains(@class,"details_tab")]//div[contains(@class, "table_section")]//text()'
        XPATH_NAME = '//h1[@property="name"]//text()'
        XPATH_HOTEL_RATING = '//span[@property="ratingValue"]//@content'
        XPATH_REVIEWS = '//a[@property="reviewCount"]/@content'
        XPATH_RANK = '//div[@class="slim_ranking"]//text()'
#         XPATH_OFFICIAL_DESCRIPTION = '//div[contains(@class,"additional_info")]//span[contains(@class,"tabs_descriptive_text")]//text()'
#         XPATH_OFFICIAL_DESCRIPTION = '//div[@class="additional_info"]//div'
        XPATH_LATITUDE = '//div[@class="mapContainer"]//@data-lat'
        XPATH_LONGITUDE = '//div[@class="mapContainer"]//@data-lng'
                                        
        ratings = parser.xpath(XPATH_RATING)
        travelers = parser.xpath(XPATH_TRAVELER)
        times = parser.xpath(XPATH_TIME)
        languages = parser.xpath(XPATH_LANGUAGE)
	
        raw_name = parser.xpath(XPATH_NAME)
        raw_rank = parser.xpath(XPATH_RANK)
        raw_review_count = parser.xpath(XPATH_REVIEWS)
        raw_rating = parser.xpath(XPATH_HOTEL_RATING)
#         raw_official_description = parser.xpath(XPATH_OFFICIAL_DESCRIPTION)
#        raw_details = parser.xpath(XPATH_DETAILS)
        raw_latitude = parser.xpath(XPATH_LATITUDE)
        raw_longitude = parser.xpath(XPATH_LONGITUDE)
					
        name = ''.join(raw_name).strip() if raw_name else None
        rank = ''.join(raw_rank).strip() if raw_rank else None
        review_count = ''.join(raw_review_count).strip() if raw_review_count else None
        hotel_rating = ''.join(raw_rating).strip() if raw_rating else None
#         official_description = ' '.join(' '.join(raw_official_description).split()) if raw_official_description else None
#        details = ' '.join(' '.join(raw_details).split()) if raw_details else None
        latitude = ' '.join(' '.join(raw_latitude).split()) if raw_latitude else None  
        longitude = ' '.join(' '.join(raw_longitude).split()) if raw_longitude else None 

        ratings_dict = OrderedDict()
        for rating in ratings:
            XPATH_RATING_KEY = './/div[@class="row_label"]//text()'
            XPATH_RATING_VALUE = './/span[@class="row_bar"]/following-sibling::span//text()'
            raw_rating_key = rating.xpath(XPATH_RATING_KEY)
            raw_rating_value = rating.xpath(XPATH_RATING_VALUE)
            cleaned_rating_key = ''.join(raw_rating_key).replace('\n','')
            cleaned_rating_value = ''.join(raw_rating_value).replace('\n','')
            ratings_dict.update({cleaned_rating_key:cleaned_rating_value})
    
        travelers_list = [] 
        for traveler in travelers:
            XPATH_TRAVELER_TYPE = './/label//text()'
            XPATH_TRAVELER_COUNT = './/span//text()'
            raw_traveler_type = traveler.xpath(XPATH_TRAVELER_TYPE)
            raw_traveler_count = traveler.xpath(XPATH_TRAVELER_COUNT)
            cleaned_traveler_type = ''.join(raw_traveler_type).replace('\n','').split(' ')
            cleaned_traveler_type = cleaned_traveler_type[0]
            cleaned_traveler_count = ''.join(raw_traveler_count).replace('\n','')
            travelers_list.append((cleaned_traveler_type,cleaned_traveler_count))
        
        times_list = []
        for time in times:
            XPATH_TIME_TYPE = './/label//text()'
            XPATH_TIME_COUNT = './/span//text()'
            raw_time_type = time.xpath(XPATH_TIME_TYPE)
            raw_time_count = time.xpath(XPATH_TIME_COUNT)
            cleaned_time_type = ''.join(raw_time_type).replace('\n','').split(' ')
            cleaned_time_type = cleaned_time_type[0]
            cleaned_time_count = ''.join(raw_time_count).replace('\n','')
            times_list.append((cleaned_time_type, cleaned_time_count))
        
        language_list = []
        for language in languages:
            XPATH_LANGUAGE_TYPE = './/label//text()'
            XPATH_LANGUAGE_COUNT = './/span//text()'
            raw_language_type = language.xpath(XPATH_LANGUAGE_TYPE)
            raw_language_count = language.xpath(XPATH_LANGUAGE_COUNT)
            cleaned_language_type = ''.join(raw_language_type).replace('\n','').split(' ')
            cleaned_language_type = cleaned_language_type[0]
            cleaned_language_count = ''.join(raw_language_count).replace('\n','')
            language_list.append(cleaned_language_type)
        language_list = language_list[1:]


        data = {'name':name,
				'rank':rank,
				'rating':hotel_rating,
                'review_count':review_count,
                'latitude':latitude,
                'longitude':longitude,
#                 'official_description':official_description,
                  'travelers':travelers_list,
                  'times of year':times_list,
                  'languages':language_list
#                  'details':details
	    }
        restaurants.append(data)
        
        
#    columns = ['property_title', 'rank', 'rating', 'review_count', 'location', 'official_description', 'travelers', 'times', 'langauges']
    df = pd.DataFrame(restaurants)
    return df



input_file = open('links.csv', 'r')
links = []
for i in input_file:
    links.append('https://www.tripadvisor.com'+i)
scraped_data = parse(links)
scraped_data
with open('restaurants_details5.csv', 'w') as csvfile:
    scraped_data.to_csv(csvfile,columns = ['latitude','longitude','name'], encoding='utf-8')
scraped_data.dropna()

,languages,latitude,longitude,name,rank,rating,review_count,times of year,travelers
0,"[English(233), Spanish(20), French(16), ]",31.952438,35.937958,Pizza Roma Cafe,#1 of 550 Restaurants in Amman,4.5,305,"[(Mar-May, (67)), (Jun-Aug, (39)), (Sep-Nov, (...","[(Families, (24)), (Couples, (58)), (Solo, (56..."
1,"[English(261), Arabic(4), French(2), ]",31.963032,35.89816,Brisket,#2 of 550 Restaurants in Amman,4.5,270,"[(Mar-May, (56)), (Jun-Aug, (58)), (Sep-Nov, (...","[(Families, (56)), (Couples, (24)), (Solo, (18..."
2,"[English(538), Arabic(40), Chinese, ]",31.951878,35.9332,Habibah Sweets,#3 of 550 Restaurants in Amman,4.5,674,"[(Mar-May, (158)), (Jun-Aug, (119)), (Sep-Nov,...","[(Families, (150)), (Couples, (74)), (Solo, (4..."
3,"[English(677), Italian(31), Arabic(28), ]",31.952278,35.92017,Fakhreldin Restaurant,#4 of 550 Restaurants in Amman,4.5,790,"[(Mar-May, (203)), (Jun-Aug, (146)), (Sep-Nov,...","[(Families, (165)), (Couples, (79)), (Solo, (1..."
4,"[English(606), Italian(33), Chinese, ]",31.94924,35.929058,Sufra Restaurant,#5 of 550 Restaurants in Amman,4.5,705,"[(Mar-May, (181)), (Jun-Aug, (134)), (Sep-Nov,...","[(Families, (126)), (Couples, (88)), (Solo, (2..."
5,"[English(460), Italian(79), Spanish(56), ]",31.992243,35.866783,Tawaheen al-Hawa,#6 of 550 Restaurants in Amman,4.5,713,"[(Mar-May, (133)), (Jun-Aug, (89)), (Sep-Nov, ...","[(Families, (111)), (Couples, (77)), (Solo, (9..."
6,"[English(273), French(4), Italian(3), ]",31.952156,35.91342,Chestnut Restaurant & Pub,#8 of 550 Restaurants in Amman,4.5,284,"[(Mar-May, (80)), (Jun-Aug, (57)), (Sep-Nov, (...","[(Families, (21)), (Couples, (26)), (Solo, (9)..."
7,"[English(59), Japanese(1)]",31.961784,35.8701,V Lounge & Restaurant,#7 of 550 Restaurants in Amman,5,60,"[(Mar-May, (55)), (Jun-Aug, (2)), (Sep-Nov, (1...","[(Families, (9)), (Couples, (7)), (Solo, (3)),..."
8,"[English(1,044), Italian(109), French(46), ]",31.952234,35.93188,Hashem,#9 of 550 Restaurants in Amman,4.5,1373,"[(Mar-May, (296)), (Jun-Aug, (226)), (Sep-Nov,...","[(Families, (167)), (Couples, (219)), (Solo, (..."
9,"[English(81), French(2), Italian(2), ]",31.953451,35.884483,Melograno,#11 of 550 Restaurants in Amman,4.5,86,"[(Mar-May, (29)), (Jun-Aug, (7)), (Sep-Nov, (2...","[(Families, (21)), (Couples, (14)), (Solo, (2)..."


In [2]:
scraped_data2 = pd.DataFrame(scraped_data.dropna())

In [3]:
lat = [(float(x)) for x in scraped_data2['latitude']]
long = [(float(x)) for x in scraped_data2['longitude']]

In [5]:
import sklearn
import sklearn.cluster
import numpy as np
matrix = scraped_data2[['latitude','longitude']].as_matrix()

In [50]:
matrix.astype(np.float)

array([[ 31.952438,  35.937958],
       [ 31.963032,  35.89816 ],
       [ 31.951878,  35.9332  ],
       ..., 
       [ 31.973347,  35.90616 ],
       [ 31.973473,  35.89811 ],
       [ 31.952627,  35.93221 ]])

In [53]:
kmeans = KMeans(init='k-means++', n_clusters=15, n_init=10)
kmeans.fit_predict(matrix)
centroids = kmeans.cluster_centers_
labels = kmeans.labels_
error = kmeans.inertia_

print("The total error of the clustering is: ", error)
print('\nCluster labels')
print(labels)
print('\n Cluster Centroids')
#print centroids

The total error of the clustering is:  0.00187027814642

Cluster labels
[11  8  4  0 10  1  0  6  4  3  1 10  9  4  7  0  2  7 12  0  4  4  6  1  7
  8 13 14  5  4 11  8  4  0 10  1  0  6  4  3  1 10  9  4  7  0  2  7 12  0
  4  4  6  1  7  8 13 14  5  4 11  8  4  0 10  1  0  6  4  3  1 10  9  4  7
  0  2  7 12  0  4  4  6  1  7  8 13 14  5  4 11  8  4  0 10  1  0  6  4  3
  1 10  9  4  7  0  2  7 12  0  4  4  6  1  7  8 13 14  5  4 11  8  4  0 10
  1  0  6  4  3  1 10  9  4  7  0  2  7 12  0  4  4  6  1  7  8 13 14  5  4
 11  8  4  0 10  1  0  6  4  3  1 10  9  4  7  0  2  7 12  0  4  4  6  1  7
  8 13 14  5  4 11  8  4  0 10  1  0  6  4  3  1 10  9  4  7  0  2  7 12  0
  4  4  6  1  7  8 13 14  5  4 11  8  4  0 10  1  0  6  4  3  1 10  9  4  7
  0  2  7 12  0  4  4  6  1  7  8 13 14  5  4 11  8  4  0 10  1  0  6  4  3
  1 10  9  4  7  0  2  7 12  0  4  4  6  1  7  8 13 14  5  4 11  8  4  0 10
  1  0  6  4  3  1 10  9  4  7  0  2  7 12  0  4  4  6  1  7  8 13 14  5  4
 11  8  4  0 10 

In [6]:
from collections import Counter
import re
#def convert(string):
#    return Counter(re.findall('[A-Z][a-z]*', string))
import ast
languages_dictsList = []
for i in scraped_data2['languages']:
    lang = (re.findall('[A-Z][a-z]*', str(i)))
    freq = (re.findall('[1-9][0-9]*', str(i)))
    dictionary = dict(zip(lang, freq))
    languages_dictsList.append(dictionary)
lang_df = pd.DataFrame(languages_dictsList)
new_names = [(i,'lang_'+i) for i in lang_df.iloc[:, 0:].columns.values]
lang_df.rename(columns = dict(new_names), inplace=True) #adding prefix to know what type of cat
resulting_df = scraped_data2.join(lang_df)
resulting_df.fillna(value=0, inplace=True) #final dataframe with languages of reviews as categories

In [7]:
travelers_dictsList = []
for i in scraped_data2['travelers']:
    trav = (re.findall('[A-Z][a-z]*', str(i)))
    freq = (re.findall('[1-9][0-9]*', str(i)))
    dictionary = dict(zip(trav, freq))
    travelers_dictsList.append(dictionary)
travelers_df = pd.DataFrame(travelers_dictsList)
new_names = [(i,'travType_'+i) for i in travelers_df.iloc[:, 0:].columns.values] 
travelers_df.rename(columns = dict(new_names), inplace=True) #adding prefix to know what type of cat
resulting_df2 = resulting_df.join(travelers_df)
resulting_df2.fillna(value=0, inplace=True) #types of travelers added as categories 

In [8]:
trav_typedf = resulting_df2[['travType_Business','travType_Couples','travType_Families','travType_Friends','travType_Solo']]

In [9]:
from sklearn.feature_extraction import DictVectorizer

In [18]:
trav_typedf.T.to_dict(orient='dict').values()
v = list(trav_typedf.T.to_dict(orient='dict').values())

In [21]:
vector = DictVectorizer()
X = vector.fit_transform(v).toarray()
name_of_c = vector.get_feature_names()

In [24]:
def findBest(X, minimum, maximum, algorithm, metric):
    k = list(range(minimum, maximum + 1))
    Scores = [0]*(maximum - minimum + 1)
    bestL = None
    bestS = float('-inf')
    curL = None
    curS = None
    for i in range(minimum, maximum + 1):
        curL = clustering(X, i, algorithm)
        curS = score(X, curL, metric)
        Scores[i - minimum] = curS
        if curS > bestS:
            bestS = curS
            bestL = curL
    return bestL

def score(X, labels, metric):
    try:
        return metrics.silhouette_score(X, labels, metric=metric)
    except:
        return 0

def clustering(X, k, algorithm):
    if algorithm == 'gmm':
        gmm = mixture.GMM(n_components=k, covariance_type='full')
        a = gmm.fit_predict(X)
        b = sorted(list(set(a)))
        for i in range(len(a)):
            a[i] = b.index(a[i])
        return a
    elif algorithm == 'hierarchical':
        model = AgglomerativeClustering(n_clusters=k)
        model.fit(X)
        return model.labels_
    else:
        kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10)
        kmeans.fit(X)
        return kmeans.labels_

In [25]:
def districts(locations, at, y, Best, minimum2, maximum2, algorithm, metric, name):
    ii = dict()
    labels = clustering(locations, Best, algorithm)
    FinalL = ['']*len(labels) 
    for i in range(len(labels)):
        if labels[i] in ii: ii[labels[i]] += [i]
        else:
            ii[labels[i]] = [i]
    already = []
    for j in ii:
        thisX = np.array([at[i] for i in ii[j]])
        curLocations = np.array([locations[i] for i in ii[j]])
        curLabels = findBest(curLocations, minimum2, maximum2, algorithm, metric)
        still = [[]]*len(set(curLabels))
        for k in range(len(curLabels)):
            still[curLabels[k]] += [thisX[k]]
        for l in range(len(still)):
            m = np.array(still[l]).T
            max1 = 0
            max2 = 0
            maxSum = float('-inf')
            max2sum = float('-inf')
            for n in range(len(m)):
                if n not in already:
                    if sum(m[n]) > maxSum:
                        max1 = n
                        maxSum = sum(m[n])
                    elif sum(m[n]) > max2sum:
                        max2 = n
                        max2sum = sum(m[n])
            still[l] = y[max1] + ', ' + y[max2]
            already += [max1, max2]
        for r in range(len(curLabels)):
            FinalL[ii[j][r]] = still[curLabels[r]]
    plottingCluster(locations, FinalL, gm=True, gm_name=name)

In [42]:
from sklearn.cluster import KMeans
import matplotlib
import sklearn.metrics as metrics
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering
from sklearn import mixture
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors
import numpy as np
import gmplot

def plottingCluster(points, labels, gm=False, gm_name='cluster.html'):
    Labels = list(set(labels))
    colors = list(matplotlib.colors.cnames)[::2]
    X = dict()
    for b in range(len(labels)):
        if labels[b] in X:
            X[labels[b]][0].append(points[b][0])
            X[labels[b]][1].append(points[b][1])
        else:
            X[labels[b]] = [[], []]
            X[labels[b]][0].append(points[b][0])
            X[labels[b]][1].append(points[b][1])
    plt.figure(figsize=(20, 10))
    if gm:
        gmap = gmplot.GoogleMapPlotter(points[0][0], points[0][1], 11)
        for j in X:
            gmap.scatter(X[j][0], X[j][1], colors[Labels.index(j)], marker=True)
        gmap.draw(gm_name)
    for l in X:
        plt.scatter(X[l][0], X[l][1], color=colors[Labels.index(l)], label=str(l), marker=".")
    plt.legend()


print('k means ++')
locations = np.array(matrix)
print(locations)
loclist = []
for k in locations:
    loclist.append(k)
districts(scraped_data2[['latitude','longitude']], X, name_of_c, 5, 2, 5, 'k-means++', 'cosine', 'k-means-clustering.html')

k means ++
[['31.952438' '35.937958']
 ['31.963032' '35.89816']
 ['31.951878' '35.9332']
 ..., 
 ['31.973347' '35.90616']
 ['31.973473' '35.89811']
 ['31.952627' '35.93221']]


KeyError: 0